# Marco muestral


Notebook para determinar las probabilidades por ageb dentro del marco censal 2020

In [1]:
import os

import numpy as np
import pandas as pd

Para el marco muestral vamos a usar los archivos por ageb enviados el 21 de Agosto 

In [2]:
archivos = {'agebs_urbanas' : './datos/01-ageb-censales_urbanas_en_localidades_quinquenios_contienelocalidades.csv', 
            'rel_agebs': './datos/01-relacion_localidades_ageb-urbanas.xlsx',
            'rel_locs_rural':'./datos/02-relacion_agebrural-localidades.xlsx',
            'locs_rural': './datos/02-upm_no_censales_ageb_rural_quinquenios_contieneagebsconsumadelocalidades.csv'}

muestraf = './datos/muestraXentidades.csv'
marco2020 = './datos/marco_ageb_pob_viv_2020.zip'

In [3]:
agebs = pd.read_csv(archivos['agebs_urbanas'])

In [4]:
agebs

,fid,cve_edo,nom_edo,cve_mun,nom_mun,cve_loc,nom_loc,cvegeomun,cvegeoloc,pobtot,...,p_75a79_f,p_75a79_m,p_80a84,p_80a84_f,p_80a84_m,p_85ymas,p_85ymas_f,p_85ymas_m,vivpar_hab,n_ageb
0,1,1,Aguascalientes,1,Aguascalientes,1,Aguascalientes,1001,10010001,863893,...,5843.0,4779.0,6383.0,3819.0,2564.0,5346.0,3373.0,1973.0,237622.0,308
1,39,1,Aguascalientes,1,Aguascalientes,237,Jaltomate,1001,10010237,2805,...,17.0,17.0,18.0,9.0,9.0,19.0,8.0,11.0,655.0,2
2,40,1,Aguascalientes,1,Aguascalientes,239,General José María Morelos y Pavón (Cañada Honda),1001,10010239,3822,...,18.0,17.0,23.0,10.0,13.0,24.0,16.0,8.0,912.0,2
3,51,1,Aguascalientes,1,Aguascalientes,293,Norias de Ojocaliente,1001,10010293,4476,...,6.0,13.0,19.0,7.0,12.0,13.0,3.0,10.0,970.0,2
4,61,1,Aguascalientes,1,Aguascalientes,357,Norias del Paso Hondo,1001,10010357,3362,...,9.0,12.0,9.0,3.0,6.0,6.0,2.0,4.0,738.0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5058,189231,32,Zacatecas,55,Villanueva,42,Malpaso,32055,320550042,3910,...,34.0,37.0,45.0,20.0,25.0,41.0,19.0,22.0,978.0,9
5059,189321,32,Zacatecas,56,Zacatecas,1,Zacatecas,32056,320560001,138444,...,1074.0,792.0,1191.0,724.0,467.0,974.0,601.0,373.0,38680.0,178
5060,189326,32,Zacatecas,56,Zacatecas,28,Cieneguillas,32056,320560028,2753,...,10.0,11.0,8.0,7.0,1.0,13.0,6.0,7.0,473.0,1
5061,189392,32,Zacatecas,57,Trancoso,1,Trancoso,32057,320570001,17021,...,90.0,85.0,123.0,68.0,55.0,127.0,58.0,69.0,3692.0,63


In [5]:
estados = agebs[['cve_edo','nom_edo']].drop_duplicates()
edos_idx = dict(zip(estados['cve_edo'], estados['nom_edo']))
idx_edos = dict(zip(estados['nom_edo'], estados['cve_edo']))
edos_idx

{1: 'Aguascalientes',
 2: 'Baja California',
 3: 'Baja California Sur',
 4: 'Campeche',
 5: 'Coahuila de Zaragoza',
 6: 'Colima',
 7: 'Chiapas',
 8: 'Chihuahua',
 9: 'Ciudad de México',
 10: 'Durango',
 11: 'Guanajuato',
 12: 'Guerrero',
 13: 'Hidalgo',
 14: 'Jalisco',
 15: 'México',
 16: 'Michoacán de Ocampo',
 17: 'Morelos',
 18: 'Nayarit',
 19: 'Nuevo León',
 20: 'Oaxaca',
 21: 'Puebla',
 22: 'Querétaro',
 23: 'Quintana Roo',
 24: 'San Luis Potosí',
 25: 'Sinaloa',
 26: 'Sonora',
 27: 'Tabasco',
 28: 'Tamaulipas',
 29: 'Tlaxcala',
 30: 'Veracruz de Ignacio de la Llave',
 31: 'Yucatán',
 32: 'Zacatecas'}

El marco censal 2020 es el siguiente

In [6]:
marco = pd.read_csv(marco2020)

La población reconocida dentro de nuestro marco muestral es la siguiente:

In [7]:
marco.pobtot.sum()

126014024

Dentro de estas vamos a censurar a todas las agebs que tengan menos de 20 viviendas particulares habitables.

In [8]:
df = marco[marco.vivpar_hab>20]
df['ent_nombre'] = df.ent.map(edos_idx)
df

C:\Users\usuario\AppData\Local\Temp\ipykernel_18020\1902301254.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ent_nombre'] = df.ent.map(edos_idx)


,fid,ent,mun,loc,ageb,AGEB,pobtot,pob12ymas,vivpar_hab,prom_ocup,estrato,ent_nombre
0,1,1,1,1,0017,0100100010017,2237,1579.0,633,3.45,3,Aguascalientes
1,2,1,1,1,006A,010010001006A,1411,1162.0,368,3.53,3,Aguascalientes
2,3,1,1,1,0106,0100100010106,2962,2238.0,991,2.88,3,Aguascalientes
3,4,1,1,1,0163,0100100010163,2698,1768.0,752,3.42,3,Aguascalientes
4,5,1,1,1,0182,0100100010182,2218,1874.0,610,3.35,3,Aguascalientes
...,...,...,...,...,...,...,...,...,...,...,...,...
81515,81516,30,142,17,0000,3014200170000,2414,1865.0,621,3.76,1,Veracruz de Ignacio de la Llave
81516,81517,30,158,16,0000,3015800160000,2140,1735.0,683,3.13,1,Veracruz de Ignacio de la Llave
81517,81518,30,160,101,0000,3016001010000,2238,1812.0,650,3.34,1,Veracruz de Ignacio de la Llave
81518,81519,30,207,83,0000,3020700830000,2131,1732.0,673,3.09,1,Veracruz de Ignacio de la Llave


In [9]:
df.estrato.isna().sum()


0

In [10]:
# vamos a calcular la población por loc para estratos 2 y 3
df['cvegeoloc'] = df.ent.astype(str).str.zfill(2) + df.mun.astype(str).str.zfill(3) + df['loc'].astype(str).str.zfill(4)

C:\Users\usuario\AppData\Local\Temp\ipykernel_18020\1912961268.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['cvegeoloc'] = df.ent.astype(str).str.zfill(2) + df.mun.astype(str).str.zfill(3) + df['loc'].astype(str).str.zfill(4)


In [11]:
df['pob_loc'] = df.groupby('cvegeoloc').pobtot.transform('sum')

C:\Users\usuario\AppData\Local\Temp\ipykernel_18020\1117655013.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['pob_loc'] = df.groupby('cvegeoloc').pobtot.transform('sum')


In [12]:
df.groupby('cvegeoloc').agg({'first'}).pob_loc.sum()

first    125557261
dtype: int64

## Tamaño muestral

Necesitamos conocer la cantidad de encuestas que se van a levantar para determinar cómo se reparten a lo largo del territorio nacional

In [13]:
# leemos los tamaños de la muestra
muestra = pd.read_csv(muestraf)
muestra['Encuestas'] = muestra['Tamaño de muestra(n)']*2

muestra.loc[7,  "Estado"] = "Coahuila de Zaragoza"
muestra.loc[10, "Estado"] = "México"
muestra.loc[15, "Estado"] = "Michoacán de Ocampo"
muestra.loc[29, "Estado"] = "Veracruz de Ignacio de la Llave"

In [14]:
# ponemos los nombres de acuerdo al dataframe
muestra['ent'] = muestra['Estado'].map(idx_edos)#.astype(int)

muestra.sort_values(by='ent')

,Estado,Drogas ilegales,alcohol_av,tabaco_av,n_di,n_alcohol,n_tabaco,Tamaño de muestra(n),Encuestas,ent
0,Aguascalientes,11.1,75.984739,23.802891,539,17,179,645,1290,1
1,Baja California,13.5,67.701021,13.365215,431,26,363,748,1496,2
2,Baja California Sur,10.9,72.986203,18.764986,550,20,242,720,1440,3
3,Campeche,11.5,69.836918,21.828594,518,24,200,655,1310,4
7,Coahuila de Zaragoza,10.8,70.196027,16.971774,556,23,274,760,1520,5
8,Colima,12.0,76.639495,16.463379,493,17,284,712,1424,6
4,Chiapas,5.8,67.103936,13.804256,1093,27,350,1288,2576,7
5,Chihuahua,11.7,78.473392,23.084595,508,15,186,624,1248,8
6,Ciudad de México,10.3,57.922512,19.009142,586,40,239,767,1534,9
9,Durango,9.5,70.598747,17.935952,641,23,256,813,1626,10


In [15]:
# cantidad de encuestas por estado
encuestas_por_estado = muestra[['Estado','Tamaño de muestra(n)']].set_index('Estado')

encuestas_por_estado.to_clipboard()

In [16]:
df[df.ent==9].groupby('estrato').vivpar_hab.sum()

estrato
1      16112
2      75389
3    2512443
Name: vivpar_hab, dtype: int64

Ahora necesitamos la proporción de agebs para determinar cómo se van a repartir todas las encuestas por estrato y estado.

In [17]:
# proporción de viviendas por estado
def escala(dfo):
    df = dfo.copy()
    return df.divide(df.sum(axis=1),axis=0)

def asigna_fila(row, colname, tabla):
    i = row['ent_nombre']
    j = row[colname]
    valor = tabla.loc[i,j]    
    return valor

# obtenemos 
viv_estrato =  df.groupby(['ent_nombre','estrato']).vivpar_hab.sum().unstack().fillna(0).astype(int)
viv_estrato_prop = escala(viv_estrato).fillna(0)
viv_estrato_prop



estrato,1,2,3
ent_nombre,,,
Aguascalientes,0.142803,0.222552,0.634645
Baja California,0.060348,0.114981,0.824671
Baja California Sur,0.081799,0.174294,0.743906
Campeche,0.237888,0.276841,0.485271
Chiapas,0.464242,0.313263,0.222495
Chihuahua,0.115491,0.126119,0.758390
Ciudad de México,0.006188,0.028952,0.964861
Coahuila de Zaragoza,0.077541,0.204117,0.718342
Colima,0.094454,0.262882,0.642664


Ahora vamos a dejar en esta variable la cuota de encuestas por vivienda por estado y estrato.

In [18]:
# cuota de viviendas a visitar por
# estrato
cuota_estrato = (viv_estrato_prop[[1,2,3]].mul(encuestas_por_estado['Tamaño de muestra(n)'],  axis=0)
                 .apply(round).astype(int))

In [19]:
cuota_estrato.sum().sum()

26016

In [20]:
prom_viv_r = 9
prom_viv_u = 12

# en este dataframe vamos a subir la cuota de agebs por 
# estrato
ks = pd.DataFrame({1:[], 2:[], 3:[]})

ks[1] = (cuota_estrato[1] / prom_viv_r).apply(round)
ks[2] = (cuota_estrato[2] / prom_viv_u).apply(round)
ks[3] = (cuota_estrato[3] / prom_viv_u).apply(round)
ks.loc['Tlaxcala',3] = 1    # ponemos una ageb en Tlax

In [21]:
ks.sum().sum()

2333

In [22]:
# cuantos agebs hay por cada entidad y estrato
# en el marco
agebs_estrato = (df.groupby(['ent_nombre','estrato']).size().unstack().fillna(0).astype(int))
agebs_estrato;

In [23]:
total_agebs = agebs_estrato.merge(ks, left_index=True, right_index=True)
total_agebs;

## Factores de expansión (1a. etapa)

Se van a calcular los factores en tres etapas para el caso urbano y para el caso rural.

La primera será un cálculo considerando los valores teóricos de número de manzanas por ageb y de viviendas por manzana.
1. Sobre las agebs

    $$
    f_a = \frac{N_e}{n_e}
    $$
    
    donde $N_e$ es la cantidad de municipios en el estrato $e$ y $n_e$ es la cantiad de agebs seleccionadas 
    
2. Sobre las agebs se seleccionaron manzanas, entonces para cada manzana seleccionada se debe construir un factor de la siguiente forma

    $$
    f_m = \frac{N_{m,a}}{n_{m,a}}
    $$
    
    donde $N_{m,a}$ es el número de manzas en la ageb $a$ y $n_{m,a}$ es el número de manzanas seleccionadas en la ageb $a$
    
3. Finalmente se genera un factor a las viviendas seleccionadas

    $$
    f_v = \frac{N_{v,m}}{n_{v,m}}
    $$
    
    donde $N_{v,m}$ es el número de viviendas por manzana mientras que $n_{v,m}$ es el número de viviendas seleccionadas en la manzana $m$.
    
El factor de expansión se calcula, entonces, como el producto de cada uno de estos factores intermedios

$$
    f_{exp} = f_a * f_m * f_v
$$

El esquema presentado es la estructura jerarquizada de composición de los factores de expansión considerando la jerarquía  de los diferentes unidades administrativas que van desde la localidad, ageb, manzana y vivienda en lo urbano

###  Agebs urbanas


Considerando la estratificación de estas unidades administrativas, primero tenemos que determinar el impacto (y por consecuencia el factor asociado a este impacto) dentro de la localidad.

In [24]:
# Totalidad de agebs a visitar
total_agebs['f_1'] = total_agebs['1_x'] / total_agebs['1_y']
total_agebs['f_2'] = total_agebs['2_x'] / total_agebs['2_y']
total_agebs['f_3'] = total_agebs['3_x'] / total_agebs['3_y']

total_agebs

,1_x,2_x,3_x,1_y,2_y,3_y,f_1,f_2,f_3
ent_nombre,,,,,,,,,
Aguascalientes,65,164,303,10,12,34,6.500000,13.666667,8.911765
Baja California,111,330,1309,5,7,51,22.200000,47.142857,25.666667
Baja California Sur,86,195,438,7,10,45,12.285714,19.500000,9.733333
Campeche,167,196,170,17,15,26,9.823529,13.066667,6.538462
Chiapas,764,1032,480,66,34,24,11.575758,30.352941,20.000000
Chihuahua,738,573,1398,8,7,39,92.250000,81.857143,35.846154
Ciudad de México,20,91,2290,1,2,62,20.000000,45.500000,36.935484
Coahuila de Zaragoza,317,580,1026,7,13,46,45.285714,44.615385,22.304348
Colima,46,152,341,7,16,38,6.571429,9.500000,8.973684


In [25]:
entprueba = df[(df.ent==1) & (df.estrato==1)]
entprueba.vivpar_hab.sum()

53460

La asignación de las probabilidades para cada ageb se va a realizar de la siguiente forma:

- Calculamos la cantidad de vivienda habitable por entidad y estrato y lo asignamos a `viviendas_e_h` 
- Asignamos la `k` 


In [26]:
pobs_viv = df.groupby(['ent_nombre','estrato']).vivpar_hab.sum().unstack().fillna(0).astype(int)

# viviendas por entidd y estrato
df['viviendas_e_h'] = df.apply(asigna_fila, axis=1, args=('estrato',pobs_viv))

# # asignamos la k
# df['k'] = df.apply(asigna_fila, axis=1, args=('estrato', ks))

# probas = (df['k'] * df['vivpar_hab']) / df['viviendas_e_h']
# df['P'] = probas

C:\Users\usuario\AppData\Local\Temp\ipykernel_18020\3609150943.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['viviendas_e_h'] = df.apply(asigna_fila, axis=1, args=('estrato',pobs_viv))


Los **factores_agebs** son para estratos 1 2 y 3

In [27]:
factores_agebs = total_agebs[['f_1','f_2','f_3']]
# probas_agebs = df.groupby(['estrato','ent_nombre']).P.sum().unstack().T.fillna(0)

In [28]:
factores_agebs['ent'] = factores_agebs.index.map(idx_edos)
factores_agebs['ent'] = factores_agebs['ent'].map(lambda x: str(x).zfill(2))
factores_agebs

C:\Users\usuario\AppData\Local\Temp\ipykernel_18020\2611847607.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  factores_agebs['ent'] = factores_agebs.index.map(idx_edos)
C:\Users\usuario\AppData\Local\Temp\ipykernel_18020\2611847607.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  factores_agebs['ent'] = factores_agebs['ent'].map(lambda x: str(x).zfill(2))


,f_1,f_2,f_3,ent
ent_nombre,,,,
Aguascalientes,6.500000,13.666667,8.911765,01
Baja California,22.200000,47.142857,25.666667,02
Baja California Sur,12.285714,19.500000,9.733333,03
Campeche,9.823529,13.066667,6.538462,04
Chiapas,11.575758,30.352941,20.000000,07
Chihuahua,92.250000,81.857143,35.846154,08
Ciudad de México,20.000000,45.500000,36.935484,09
Coahuila de Zaragoza,45.285714,44.615385,22.304348,05
Colima,6.571429,9.500000,8.973684,06


### Factores de manzanas

De la misma forma ahora vamos a calcular el impacto de las manzanas dentro de las agebs.

Para esto, podemos hacer una lista de nuestro marco separado por estratos y por cada lista podemos agregar una columna correspondiente a su factor por ageb



In [29]:
# cargamos marco con manzanas y 
# archivo de encuesta
#
fmarco_mzas = './datos/07_marco_muestral_manzana_loc.zip'
fcorte = './datos/cortes/w18/enasama_2024W18_0429_0505.pkl'
marco_mzas = pd.read_csv(fmarco_mzas)
w18 = pd.read_pickle(fcorte)

C:\Users\usuario\AppData\Local\Temp\ipykernel_18020\2023123177.py:6: DtypeWarning: Columns (2,10) have mixed types. Specify dtype option on import or set low_memory=False.
  marco_mzas = pd.read_csv(fmarco_mzas)


In [30]:
T = marco_mzas.pobtot.sum()
T

125572812

In [31]:
marco_mzas[~marco_mzas.cvegeomz.isna()].pobtot.sum(), marco_mzas[marco_mzas.cvegeomz.isna()].pobtot.sum()
marco_mzas[~marco_mzas.cvegeomz.isna()].pobtot.sum() / T, marco_mzas[marco_mzas.cvegeomz.isna()].pobtot.sum() / T

(0.7930677064076578, 0.20693229359234228)

Obtenemos los folios de la encuesta para de ahí obtener la ageb 

In [32]:
# asigna una columna a cada estrato
criterio = {1: 'f_1', 2:'f_2', 3:'f_3'}

In [33]:
# expandimos el df con estrato apilado 
# en estratos 1 2 y 3
#
factores_estrato = pd.concat((df.loc[df.estrato==1, ['AGEB','estrato','ent','pobtot','pob_loc']],
                              df.loc[df.estrato==2, ['AGEB','estrato','ent','pobtot','pob_loc']],
                              df.loc[df.estrato==3, ['AGEB','estrato','ent','pobtot','pob_loc']]))

factores_estrato['ent'] = factores_estrato['ent'].astype(str).str.zfill(2)

In [34]:
factores_estrato.sample(20)

,AGEB,estrato,ent,pobtot,pob_loc
10791,080670001074A,1,08,399,2219
66936,0800800000448,1,08,266,10338
32380,1604300010352,2,16,1748,61330
43473,2114000140450,2,21,1669,20283
80410,3109100000104,1,31,1851,4278
21756,130480001253A,3,13,1481,297544
52183,2604300011833,3,26,1875,260191
16890,1102000011813,3,11,1406,1578499
58231,3010800010630,3,30,102,101336
6175,0701200010338,2,07,212,35747


In [35]:
factores_estrato['factor_col'] = factores_estrato['estrato'].map(criterio)
merged_factores = pd.merge(factores_estrato, factores_agebs, on='ent')
merged_factores['factor_a'] = merged_factores.apply(lambda row: row[row['factor_col']], axis=1)

factores_estrato = merged_factores[['AGEB','estrato','ent','pobtot','pob_loc','factor_a']]

In [36]:
factores_estrato[factores_estrato.estrato>1]

,AGEB,estrato,ent,pobtot,pob_loc,factor_a
317,0500300010081,2,05,599,21161,44.615385
318,0500300010096,2,05,405,21161,44.615385
319,0500300010109,2,05,1051,21161,44.615385
320,0500300010113,2,05,865,21161,44.615385
321,0500300010128,2,05,917,21161,44.615385
...,...,...,...,...,...,...
61870,2700400012359,3,27,789,340018,10.888889
61871,2700400012560,3,27,2255,340018,10.888889
61872,2700400012575,3,27,3160,340018,10.888889
61873,2700400012611,3,27,3187,340018,10.888889


In [37]:
# hacemos la comprobacion por cada 
# estrato 
factores_agebs.merge(factores_estrato[factores_estrato.estrato==1].groupby('ent').factor_a.mean(), how='inner',
                    left_on='ent', right_index=True)

,f_1,f_2,f_3,ent,factor_a
ent_nombre,,,,,
Aguascalientes,6.500000,13.666667,8.911765,01,6.500000
Baja California,22.200000,47.142857,25.666667,02,22.200000
Baja California Sur,12.285714,19.500000,9.733333,03,12.285714
Campeche,9.823529,13.066667,6.538462,04,9.823529
Chiapas,11.575758,30.352941,20.000000,07,11.575758
Chihuahua,92.250000,81.857143,35.846154,08,92.250000
Ciudad de México,20.000000,45.500000,36.935484,09,20.000000
Coahuila de Zaragoza,45.285714,44.615385,22.304348,05,45.285714
Colima,6.571429,9.500000,8.973684,06,6.571429


### Manzanas

La cantidad de encuestas por manzana por ageb seleccionadas fueron de 

$$
m_{ageb} = 5
$$

Entonces con esto y el número de manzanas por ageb podemos obtener al factor correspondiente a manzanas

In [38]:
# este dataframe solo tiene las claves de ageb con estrato > 1 (2 y 3)
factores_urbanos = factores_estrato[factores_estrato.estrato>1]

In [39]:
# manzanas por ageb
mzas_x_ageb = pd.DataFrame(marco_mzas.groupby('cvegeoageb').cvegeomz.size())
mzas_x_ageb.columns = ['n']
num_mzas = 5

# factor de manzanas por ageb
mzas_x_ageb['factor_m'] = mzas_x_ageb['n'] / num_mzas
mzas_x_ageb['ent'] = mzas_x_ageb.index.str[:2]


In [40]:
# un ejemplo de cómo qudan las 
# cuentas de manzanas por ageb 
mzas_x_ageb.sample(10)

,n,factor_m,ent
cvegeoageb,,,
2115600010732,2,0.4,21
280330001026A,43,8.6,28
1604100010038,54,10.8,16
3004000010242,27,5.4,30
3010502730427,39,7.8,30
1408501730372,6,1.2,14
1412000014643,58,11.6,14
2101500010185,14,2.8,21
0401200010052,33,6.6,04


In [41]:

cols_a_usar = ['fid', 'cvegeoageb', 'cvegeomz', 'pobtot', 'p_12ymas', 'vivpar_hab',
       'ocupvivpar', 'prom_ocup', 'origen', 'tipo_selec', 'cvegeosel_m_l',
       'cvegeoloc', 'pob_ageb','pob_loc','estrato','ent']

In [42]:
marco_mzas = pd.merge(marco_mzas, 
                      df[['AGEB','estrato','ent','pobtot','pob_loc']].rename(columns={'pobtot':'pob_ageb'}), 
                      how='inner', 
                      left_on='cvegeoageb', 
                      right_on='AGEB')[cols_a_usar]


In [57]:
# generamos los factores para cada manzana ocupando el archivo 
# recién generado
factores_mzas = (marco_mzas[['cvegeoageb','cvegeomz', 'pob_ageb' ,'pob_loc', 'estrato']]
                 .merge(mzas_x_ageb, 
                        how='left', left_on='cvegeoageb', right_index=True))

In [44]:
factores_mzas[factores_mzas.cvegeomz.isna()].describe()

,pob_ageb,pob_loc,estrato,n,factor_m
count,171275.000000,171275.000000,171275.000000,171275.000000,171275.000000
mean,3881.937259,27062.160327,1.000023,36.694713,7.338943
std,4469.721961,26826.557511,0.004833,43.714136,8.742827
min,43.000000,56.000000,1.000000,1.000000,0.200000
25%,884.000000,9279.500000,1.000000,13.000000,2.600000
50%,2306.000000,18393.000000,1.000000,24.000000,4.800000
75%,5291.000000,35529.000000,1.000000,44.000000,8.800000
max,43068.000000,158809.000000,2.000000,483.000000,96.600000


In [45]:
# esto cuenta la cantidad de manzanas que no 
# tienen clave de manzana
#
factores_mzas.isna().sum()

cvegeoageb         0
cvegeomz      171275
pob_ageb           0
pob_loc            0
estrato            0
n                  0
factor_m           0
ent                0
dtype: int64

In [58]:
mzas_na = factores_mzas.cvegeomz.isna()
marco_mzas.loc[mzas_na, ['cvegeomz','origen','tipo_selec']].describe()

,cvegeomz,origen,tipo_selec
count,0,171275,171275
unique,0,3,1
top,NaN,iter,loc
freq,NaN,171094,171275


In [47]:
len(factores_mzas)

1720841

In [75]:
# aquí nos quedamos con los puros factores para 
# las claves de manzana
# 
cols_usar = ['cvegeoageb', 'cvegeomz','estrato','ent', 'factor_a','factor_m', 'pob_ageb','pob_loc']

factores_estrato_mza = pd.merge(factores_urbanos[['AGEB', 'estrato', 'ent', 'factor_a']], 
                                factores_mzas[['cvegeoageb','cvegeomz','factor_m', 'pob_ageb','pob_loc']], 
                                how='inner', 
                                left_on='AGEB', 
                                right_on='cvegeoageb')[cols_usar]
# factores_estrato_mza.rename(columns={'fm':'factor_m'})
# factores_estrato_mza = factores_estrato_mza[~factores_estrato_mza.cvegeomz.isna()]

Los factores por estrato y manzana son los siguientes

In [76]:
factores_estrato_mza

,cvegeoageb,cvegeomz,estrato,ent,factor_a,factor_m,pob_ageb,pob_loc
0,0500300010081,0500300010081034,2,05,44.615385,7.6,599,21161
1,0500300010081,0500300010081015,2,05,44.615385,7.6,599,21161
2,0500300010081,0500300010081025,2,05,44.615385,7.6,599,21161
3,0500300010081,0500300010081037,2,05,44.615385,7.6,599,21161
4,0500300010081,0500300010081035,2,05,44.615385,7.6,599,21161
...,...,...,...,...,...,...,...,...
1498318,2700400012626,2700400012626012,3,27,10.888889,5.6,4661,340018
1498319,2700400012626,2700400012626008,3,27,10.888889,5.6,4661,340018
1498320,2700400012626,2700400012626004,3,27,10.888889,5.6,4661,340018
1498321,2700400012626,2700400012626018,3,27,10.888889,5.6,4661,340018


In [249]:
factores_estrato_mza.describe(include='all')

,cvegeoageb,cvegeomz,estrato,ent,factor_a,factor_m
count,1549566,1549566,1.498319e+06,1498319,1.498319e+06,1.549566e+06
unique,50331,1549566,NaN,32,NaN,NaN
top,2056500370048,0501001760103006,NaN,15,NaN,NaN
freq,375,1,NaN,134690,NaN,NaN
mean,NaN,NaN,2.521779e+00,NaN,4.091632e+01,8.220690e+00
std,NaN,NaN,4.995256e-01,NaN,2.468855e+01,4.383419e+00
min,NaN,NaN,2.000000e+00,NaN,6.538462e+00,2.000000e-01
25%,NaN,NaN,2.000000e+00,NaN,2.504000e+01,6.000000e+00
50%,NaN,NaN,3.000000e+00,NaN,3.248649e+01,7.800000e+00
75%,NaN,NaN,3.000000e+00,NaN,5.046667e+01,9.600000e+00


In [78]:
# en este dataframe tenemos a todos las claves de manzana 
# a los que se les calculo su factor_m
#
factores_estrato_mza.sort_values(['ent','estrato'])

,cvegeoageb,cvegeomz,estrato,ent,factor_a,factor_m,pob_ageb,pob_loc
1236675,010010237458A,010010237458A033,2,01,13.666667,7.4,1998,2805
1236676,010010237458A,010010237458A034,2,01,13.666667,7.4,1998,2805
1236677,010010237458A,010010237458A035,2,01,13.666667,7.4,1998,2805
1236678,010010237458A,010010237458A036,2,01,13.666667,7.4,1998,2805
1236679,010010237458A,010010237458A030,2,01,13.666667,7.4,1998,2805
...,...,...,...,...,...,...,...,...
1236670,3205600012267,3205600012267005,3,32,17.850000,3.4,290,137320
1236671,3205600012267,3205600012267007,3,32,17.850000,3.4,290,137320
1236672,3205600012267,3205600012267013,3,32,17.850000,3.4,290,137320
1236673,3205600012267,3205600012267015,3,32,17.850000,3.4,290,137320


### Viviendas

En el caso de las viviendas tenemos que hacer lo mismo, debemos tomar las manzanas recién calculadas y revisar `vivpar_hab` para determinar cuántos viviendas particulares habitables había 

In [85]:
marco_mzas.loc[marco_mzas.cvegeomz.isin(factores_estrato_mza.cvegeomz), ['cvegeomz','vivpar_hab']]

,cvegeomz,vivpar_hab
0,0501001760103006,9.0
1,0501001760103005,6.0
2,0501001760103012,5.0
3,0501001760103009,11.0
4,0501001760103008,9.0
...,...,...
1720836,NaN,736.0
1720837,NaN,573.0
1720838,NaN,733.0
1720839,NaN,683.0



## Muestra

Cargamos la muestra y determinamos la cantidad de manzanas por ageb (urbanos) y la cantidad de viviendas por manzana. Esto se traslada a la cantidad de folios seamana -asociados a su ageb- y la cantidad de encuestas por semana -asociadas a su manzana-.

In [32]:
w18_sitios = pd.DataFrame(w18.folio.astype(str), columns=['folio'])
w18_sitios['ageb'] = w18_sitios.folio.apply(lambda x: x[:13])
w18_sitios['mza'] = w18_sitios.folio.apply(lambda x: x[:16])

In [33]:
w18_sitios['consecutivo'] = w18_sitios.groupby(['folio','mza']).cumcount() + 1
w18_sitios['folio'] = w18_sitios['folio'] + '_' + w18_sitios['consecutivo'].astype(str)
w18_sitios = w18_sitios.drop('consecutivo', axis=1)

w18_sitios

,folio,ageb,mza
0,161020001080002116003009_1,1610200010800,1610200010800021
1,161020001080002116003009_2,1610200010800,1610200010800021
2,161020001080002616003005_1,1610200010800,1610200010800026
3,161020001176200216003004_1,1610200011762,1610200011762002
4,161020001176200216003004_2,1610200011762,1610200011762002
...,...,...,...
36064,240420001012900124004401_2,2404200010129,2404200010129001
36065,240420001012900124004402_1,2404200010129,2404200010129001
36066,240420001012900124004402_2,2404200010129,2404200010129001
36067,240420001012900324004401_1,2404200010129,2404200010129003


In [34]:
# número de manzanas por ageb
mza_per_ageb = w18_sitios.groupby('ageb')['mza'].nunique().reset_index(name='mza_size_per_ageb')
print(mza_per_ageb)
folio_per_mza = w18_sitios.groupby('mza')['folio'].count().reset_index(name='folio_count_per_mza')
print(folio_per_mza)

               ageb  mza_size_per_ageb
0     0100100001763                  3
1     0100100001886                  1
2     0100100001937                  5
3     0100100010303                  5
4     0100100010341                  6
...             ...                ...
2347  3205600010595                  5
2348  3205600010970                  5
2349  3205600010985                  1
2350  3205600011038                  5
2351  3205600011150                  6

[2352 rows x 2 columns]
                    mza  folio_count_per_mza
0      0100100001763183                    3
1      0100100001763206                    6
2      0100100001763315                    8
3      0100100001886429                   24
4      0100100001937179                    5
...                 ...                  ...
10047  3205600011150003                    5
10048  3205600011150004                    6
10049  3205600011150012                    1
10050  3205600011150027                    5
10051  32056

In [35]:
mzas_muestra = w18_sitios['ageb'].unique()
mzas_muestra, len(mzas_muestra)

(array(['1610200010800', '1610200011762', '0100100001937', ...,
        '0601000011017', '0706201360024', '2404200010129'], dtype=object),
 2352)

In [65]:
sitios_merged = pd.merge(w18_sitios, mza_per_ageb, on='ageb', how='left')
sitios_merged = pd.merge(sitios_merged, folio_per_mza, on='mza', how='left').set_index(['folio'])
sitios_merged

,ageb,mza,mza_size_per_ageb,folio_count_per_mza
folio,,,,
161020001080002116003009_1,1610200010800,1610200010800021,3,8
161020001080002116003009_2,1610200010800,1610200010800021,3,8
161020001080002616003005_1,1610200010800,1610200010800026,3,6
161020001176200216003004_1,1610200011762,1610200011762002,2,4
161020001176200216003004_2,1610200011762,1610200011762002,2,4
...,...,...,...,...
240420001012900124004401_2,2404200010129,2404200010129001,2,4
240420001012900124004402_1,2404200010129,2404200010129001,2,4
240420001012900124004402_2,2404200010129,2404200010129001,2,4


In [37]:
sitios_merged.loc['1610200010800','1610200010800021','161020001080002116003009_1']

mza_size_per_ageb      3
folio_count_per_mza    8
Name: (1610200010800, 1610200010800021, 161020001080002116003009_1), dtype: int64

In [75]:
# manzanas en la muestra dentro
# del marco
#
bb = mzas[mzas.index.isin(mzas_muestra)]
bb

,n,fm,ent
cvegeoageb,,,
0100100001763,39,7.8,01
0100100001886,16,3.2,01
0100100001937,44,8.8,01
0100100010303,25,5.0,01
0100100010341,24,4.8,01
...,...,...,...
3205600010595,47,9.4,32
3205600010970,6,1.2,32
3205600010985,6,1.2,32


In [78]:
bb[bb.ent=='01'].sample(3)

,n,fm,ent
cvegeoageb,,,
0100100013562,2,0.4,01
0100100012329,51,10.2,01
0100700300148,28,5.6,01


In [76]:
# aqui queremos extender el dataframe de mzas_factor
# para incorporar las manzanas 
mzas_factor = pd.DataFrame(mzas.merge(mzas_selec, how='right', left_index=True, right_index=True), )
# mzas_factor. columns = ['marco','enasama', 'ent']
mzas_factor

NameError: name 'mzas_selec' is not defined

### Tercer etapa: factores de viviendas

La última etapa de selección es sobre las viviendas donde se seleccionaron 

$$
\begin{align}
{\text{prom_viv_r}} &= 9 \\
{\text{prom_viv_u}} &= 12
\end{align}
$$

Entonces para las manzanas seleccionadas tenemos que 

In [317]:
prom_viv_r, prom_viv_u = 9, 12

In [318]:
mzas

,n,fm,ent
cvegeoageb,,,
0100100001068,28,5.6,01
0100100001123,25,5.0,01
0100100001138,52,10.4,01
0100100001176,3,0.6,01
0100100001180,4,0.8,01
...,...,...,...
3205800010087,1,0.2,32
3205800010091,1,0.2,32
3205800010104,2,0.4,32


In [345]:
# vamos a seleccionar a las manzanas urbanas
mzas_urbanas = marco_mzas.loc[marco_mzas.tipo_selec=='mza',['cvegeoageb','cvegeomz','vivpar_hab','ocupvivpar']].set_index('cvegeomz')
mzas_urbanas

,cvegeoageb,vivpar_hab,ocupvivpar
cvegeomz,,,
0501001760103006,0501001760103,9.0,28.0
0501001760103005,0501001760103,6.0,25.0
0501001760103012,0501001760103,5.0,20.0
0501001760103009,0501001760103,11.0,37.0
0501001760103008,0501001760103,9.0,33.0
...,...,...,...
2201400013399002,2201400013399,22.0,72.0
2201400013187011,2201400013187,34.0,115.0
2201400013666007,2201400013666,17.0,77.0


Usando estas manzanas y la cantidad de `vivpar_hab` en cada una, es que podemos calcular los factores de vivienda correspondientes a cada una

In [346]:
mzas_urbanas['fv'] = mzas_urbanas['vivpar_hab'] / prom_viv_u
mzas_urbanas

,cvegeoageb,vivpar_hab,ocupvivpar,fv
cvegeomz,,,,
0501001760103006,0501001760103,9.0,28.0,0.750000
0501001760103005,0501001760103,6.0,25.0,0.500000
0501001760103012,0501001760103,5.0,20.0,0.416667
0501001760103009,0501001760103,11.0,37.0,0.916667
0501001760103008,0501001760103,9.0,33.0,0.750000
...,...,...,...,...
2201400013399002,2201400013399,22.0,72.0,1.833333
2201400013187011,2201400013187,34.0,115.0,2.833333
2201400013666007,2201400013666,17.0,77.0,1.416667


### Reconstrucción de factores


Con estos factores calculados ahora podemos hacer la reconstrucción


Tomamos una ageb de nuestra muestra 

1. Tomamos las manzanas en nuestra muestra para ver qué factor de viviendas tienen

In [322]:
mzas_muestra, len(mzas_muestra), len(mzas_muestra[0])

(array(['1610200010800', '1610200011762', '0100100001937', ...,
        '0601000011017', '0706201360024', '2404200010129'], dtype=object),
 2352,
 13)

In [323]:
len(mzas_urbanas.iloc[0].name)

16

In [339]:
# mzas_muestra son las manzanas que tenemos en la
# muestra y hay que ubicarlas en el cálculo de
# 
mzas_ageb =  mzas[mzas.index.isin(mzas_muestra)]
mzas_ageb

,n,fm,ent
cvegeoageb,,,
0100100001763,39,7.8,01
0100100001886,16,3.2,01
0100100001937,44,8.8,01
0100100010303,25,5.0,01
0100100010341,24,4.8,01
...,...,...,...
3205600010595,47,9.4,32
3205600010970,6,1.2,32
3205600010985,6,1.2,32


Vamos a agregar el factor de expansión por estado 

C:\Users\usuario\AppData\Local\Temp\ipykernel_4604\2611847607.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  factores_agebs['ent'] = factores_agebs.index.map(idx_edos)
C:\Users\usuario\AppData\Local\Temp\ipykernel_4604\2611847607.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  factores_agebs['ent'] = factores_agebs['ent'].map(lambda x: str(x).zfill(2))


,f_1,f_2,f_3,ent
ent_nombre,,,,
Aguascalientes,6.500000,13.666667,8.911765,01
Baja California,22.200000,47.142857,25.666667,02
Baja California Sur,12.285714,19.500000,9.733333,03
Campeche,9.823529,13.066667,6.538462,04
Chiapas,11.575758,30.352941,20.000000,07
Chihuahua,92.250000,81.857143,35.846154,08
Ciudad de México,20.000000,45.500000,36.935484,09
Coahuila de Zaragoza,45.285714,44.615385,22.304348,05
Colima,6.571429,9.500000,8.973684,06


In [341]:
# hasta este momento se tienen los factores a nivel manzana 
# hay que separar las mzas_agebs en su estrato para juntarla 
# con las f_2 o f_3 como corresponda
#
mzas_ageb = mzas_ageb.reset_index().merge(factores_agebs[['f_2','f_3','ent']], how='right', on='ent')
mzas_ageb

,cvegeoageb,n,fm,ent,f_2,f_3
0,0100100001763,39,7.8,01,13.666667,8.911765
1,0100100001886,16,3.2,01,13.666667,8.911765
2,0100100001937,44,8.8,01,13.666667,8.911765
3,0100100010303,25,5.0,01,13.666667,8.911765
4,0100100010341,24,4.8,01,13.666667,8.911765
...,...,...,...,...,...,...
2113,3205600010595,47,9.4,32,27.608696,17.850000
2114,3205600010970,6,1.2,32,27.608696,17.850000
2115,3205600010985,6,1.2,32,27.608696,17.850000
2116,3205600011038,11,2.2,32,27.608696,17.850000


In [350]:
factores_urbanos = mzas_urbanas.reset_index().merge(mzas_ageb, how='right', on='cvegeoageb').dropna()

In [361]:
factores_urbanos

,cvegeomz,cvegeoageb,vivpar_hab,ocupvivpar,fv,n,fm,ent,f_2,f_3
3,0100100010303025,0100100010303,6.0,21.0,0.500000,25,5.0,01,13.666667,8.911765
4,0100100010303026,0100100010303,21.0,78.0,1.750000,25,5.0,01,13.666667,8.911765
5,0100100010303027,0100100010303,14.0,38.0,1.166667,25,5.0,01,13.666667,8.911765
6,0100100010303028,0100100010303,0.0,0.0,0.000000,25,5.0,01,13.666667,8.911765
7,0100100010303018,0100100010303,42.0,138.0,3.500000,25,5.0,01,13.666667,8.911765
...,...,...,...,...,...,...,...,...,...,...
58700,3205600011150010,3205600011150,42.0,187.0,3.500000,29,5.8,32,27.608696,17.850000
58701,3205600011150009,3205600011150,12.0,58.0,1.000000,29,5.8,32,27.608696,17.850000
58702,3205600011150011,3205600011150,3.0,13.0,0.250000,29,5.8,32,27.608696,17.850000
58703,3205600011150031,3205600011150,0.0,0.0,0.000000,29,5.8,32,27.608696,17.850000
